In [2]:
# Verificar e instalar librerías si es necesario
try:
    import numpy as np
    print("✓ numpy instalado")
except ImportError:
    !pip install numpy
    import numpy as np

try:
    import pandas as pd
    print("✓ pandas instalado")
except ImportError:
    !pip install pandas
    import pandas as pd

try:
    from scipy import stats
    print("✓ scipy instalado")
except ImportError:
    !pip install scipy
    from scipy import stats

try:
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    print("✓ statsmodels instalado")
except ImportError:
    !pip install statsmodels
    import statsmodels.api as sm
    import statsmodels.formula.api as smf

try:
    from sklearn.linear_model import LassoCV
    from sklearn.preprocessing import StandardScaler
    print("✓ scikit-learn instalado")
except ImportError:
    !pip install scikit-learn
    from sklearn.linear_model import LassoCV
    from sklearn.preprocessing import StandardScaler

print("¡Todas las librerías están listas!")

✓ numpy instalado
✓ pandas instalado
✓ scipy instalado
✓ statsmodels instalado
✓ scikit-learn instalado
¡Todas las librerías están listas!


# 3.1 DATA SIMULATION

In [3]:
# Configurar semilla para reproducibilidad
np.random.seed(42)
# 1. Generar covariables (n = 1000 individuos)
n = 1000
# X1 y X3 continuas ~ N(0,1)
X1 = np.random.normal(0,1,n)
X3 = np.random.normal(0,1,n)
# X2 y X4 binarias ~ Bernoulli(0.5)
X2 = np.random.binomial(1,0.5,n)
X4 = np.random.binomial(1,0.5,n)
# 2. Tratamiento D ~ Bernoulli(0.5)
D = np.random.binomial(1,0.5,n)
# Error aleatorio epsilon ~ N(0,1)
epsilon = np.random.normal(0,1,n)
# 3. VARIABLE OUTCOME Y según la fórmula del lab 3:
Y = 2*D + 0.5*X1 - 0.3*X2 + 0.2*X3 + epsilon
# Note que es el proceso generador de datos. Y no depende de X4 lo que nos
#lleva a intuir que X4 no es una variable relevante.

In [4]:
# 4. Crear DataFrame
df = pd.DataFrame({
    'Y': Y,
    'D': D,
    'X1': X1,
    'X2': X2,
    'X3': X3,
    'X4': X4
})

print("=" * 60)
print("3.1 DATA SIMULATION - COMPLETADO")
print("=" * 60)
print("Primeras 5 filas del DataFrame:")
print(df.head())
print(f"\nDimensión del DataFrame: {df.shape}")

3.1 DATA SIMULATION - COMPLETADO
Primeras 5 filas del DataFrame:
          Y  D        X1  X2        X3  X4
0  1.920756  1  0.496714   0  1.399355   0
1  1.711433  1 -0.138264   0  0.924634   1
2  1.649491  1  0.647689   0  0.059630   1
3 -0.800544  0  1.523030   0 -0.646937   0
4 -0.131596  0 -0.234153   1  0.698223   1

Dimensión del DataFrame: (1000, 6)


In [5]:
# Balance check (comparación de medias)
tratamiento = df[df['D'] == 1]
control = df[df['D'] == 0]

In [6]:
def balance_check(variable):
    t_stat, p_value = stats.ttest_ind(tratamiento[variable], control[variable])
    print(f"{variable}:")
    print(f"  Media tratamiento: {tratamiento[variable].mean():.3f}")
    print(f"  Media control: {control[variable].mean():.3f}")
    print(f"  Diferencia: {tratamiento[variable].mean() - control[variable].mean():.3f}")
    print(f"  p-valor: {p_value:.3f}")
    print("-" * 40)

    print("\nBALANCE CHECK - Comparación de medias")
for cov in ['X1', 'X2', 'X3', 'X4']:
    balance_check(cov)

X1:
  Media tratamiento: 0.049
  Media control: -0.009
  Diferencia: 0.058
  p-valor: 0.350
----------------------------------------

BALANCE CHECK - Comparación de medias
X2:
  Media tratamiento: 0.510
  Media control: 0.496
  Diferencia: 0.014
  p-valor: 0.655
----------------------------------------

BALANCE CHECK - Comparación de medias
X3:
  Media tratamiento: 0.077
  Media control: 0.065
  Diferencia: 0.013
  p-valor: 0.843
----------------------------------------

BALANCE CHECK - Comparación de medias
X4:
  Media tratamiento: 0.465
  Media control: 0.500
  Diferencia: -0.035
  p-valor: 0.271
----------------------------------------

BALANCE CHECK - Comparación de medias


# 3.2 ESTIMATING THE AVERAGE TREATMENT EFFECT

In [7]:
# 1. Regresión simple: Y ~ D
modelo_simple = smf.ols('Y ~ D', data=df).fit()
print("=== REGRESIÓN SIMPLE (Y ~ D) ===")
print("Coeficiente de D (ATE):", round(modelo_simple.params['D'], 3))
print("Error estándar:", round(modelo_simple.bse['D'], 3))
print()

=== REGRESIÓN SIMPLE (Y ~ D) ===
Coeficiente de D (ATE): 2.087
Error estándar: 0.072



In [8]:
# 2. Regresión con controles: Y ~ D + X1 + X2 + X3 + X4
modelo_completo = smf.ols('Y ~ D + X1 + X2 + X3 + X4', data=df).fit()
print("=== REGRESIÓN CON CONTROLES (Y ~ D + X1 + X2 + X3 + X4) ===")
print("Coeficiente de D (ATE):", round(modelo_completo.params['D'], 3))
print("Error estándar:", round(modelo_completo.bse['D'], 3))
print()

=== REGRESIÓN CON CONTROLES (Y ~ D + X1 + X2 + X3 + X4) ===
Coeficiente de D (ATE): 2.059
Error estándar: 0.062



In [9]:
# 3. Comparación de resultados
print("=== COMPARACIÓN ===")
print(f"Diferencia en ATE: {modelo_completo.params['D'] - modelo_simple.params['D']:.3f}")
print(f"Diferencia en SE: {modelo_completo.bse['D'] - modelo_simple.bse['D']:.3f}")

=== COMPARACIÓN ===
Diferencia en ATE: -0.028
Diferencia en SE: -0.010


# 3.3 LASSO AND VARIABLE SELECTION

In [12]:
# =============================================================================
# 3.3 LASSO CON PARÁMETROS TEÓRICOS (Enfoque Chernozhukov)
# =============================================================================

print("\n" + "=" * 70)
print("3.3 LASSO CON PARÁMETROS TEÓRICOS (Enfoque Chernozhukov et al.)")
print("=" * 70)

from sklearn.linear_model import Lasso



3.3 LASSO CON PARÁMETROS TEÓRICOS (Enfoque Chernozhukov et al.)


In [35]:
# =============================================================================
# 3.3 LASSO CON PARÁMETROS TEÓRICOS (Enfoque Chernozhukov) - CORREGIDO
# =============================================================================

print("\n" + "=" * 70)
print("3.3 LASSO CON PARÁMETROS TEÓRICOS (Enfoque Chernozhukov et al.)")
print("=" * 70)

from sklearn.linear_model import Lasso, LassoCV
import numpy as np
from scipy import stats

# 1. Calcular lambda teórico recomendado
def chernozhukov_lambda(n, p, confidence=0.95):
    """
    Calcula el parámetro de penalización óptimo según Chernozhukov et al.
    
    Parameters:
    n: tamaño de muestra
    p: número de covariables
    confidence: nivel de confianza (0.95 para 95%)
    """
    c = 1.1  # Constante empíricamente determinada
    alpha = 1 - confidence
    lambda_opt = c * np.sqrt(n) * stats.norm.ppf(1 - alpha / (2 * p))
    return lambda_opt

# Preparar datos
X_cov = df[['X1', 'X2', 'X3', 'X4']]
y = df['Y']

# Preprocesamiento adecuado (solo estandarizar continuas)
X_preprocessed = X_cov.copy()
scaler = StandardScaler()
X_preprocessed[['X1', 'X3']] = scaler.fit_transform(X_cov[['X1', 'X3']])

# Calcular para nuestro caso
n = len(df)
p = 4  # X1, X2, X3, X4
lambda_chernozhukov = chernozhukov_lambda(n, p)

print(f"Tamaño de muestra (n): {n}")
print(f"Número de covariables (p): {p}")
print(f"Lambda teórico Chernozhukov (crudo): {lambda_chernozhukov:.6f}")
print(f"Lambda para sklearn (dividido por n): {lambda_chernozhukov/n:.6f}")
print()

# 2. Aplicar LASSO con el lambda teórico
lasso_theoretical = Lasso(alpha=lambda_chernozhukov/n,  # ¡Importante: dividir por n!
                         random_state=42,
                         max_iter=10000)

lasso_theoretical.fit(X_preprocessed, y)

# 3. Aplicar también LASSO con validación cruzada para comparar
print("Ajustando LASSO con validación cruzada...")
lasso_cv = LassoCV(cv=5, random_state=42, n_alphas=100)
lasso_cv.fit(X_preprocessed, y)

print("✓ LASSO con validación cruzada completado")
print()

# 4. Analizar resultados teóricos
coeficientes_theoretical = pd.DataFrame({
    'Variable': X_cov.columns,
    'Coeficiente': lasso_theoretical.coef_,
    'Abs_Coeficiente': np.abs(lasso_theoretical.coef_)
}).sort_values('Abs_Coeficiente', ascending=False)

# Variables seleccionadas
umbral = 1e-6  # Casi cero
variables_seleccionadas_theoretical = coeficientes_theoretical[
    coeficientes_theoretical['Abs_Coeficiente'] > umbral
]['Variable'].tolist()

print("=== RESULTADOS LASSO TEÓRICO ===")
print("Coeficientes con lambda de Chernozhukov:")
print(coeficientes_theoretical)
print()
print(f"Variables seleccionadas: {variables_seleccionadas_theoretical}")
print(f"Variables eliminadas: {list(set(X_cov.columns) - set(variables_seleccionadas_theoretical))}")
print()

# 5. Analizar resultados de validación cruzada
coeficientes_cv = pd.DataFrame({
    'Variable': X_cov.columns,
    'Coeficiente': lasso_cv.coef_,
    'Abs_Coeficiente': np.abs(lasso_cv.coef_)
}).sort_values('Abs_Coeficiente', ascending=False)

variables_seleccionadas_cv = coeficientes_cv[
    coeficientes_cv['Abs_Coeficiente'] > umbral
]['Variable'].tolist()

print("=== RESULTADOS VALIDACIÓN CRUZADA ===")
print("Coeficientes con validación cruzada:")
print(coeficientes_cv)
print()
print(f"Variables seleccionadas (CV): {variables_seleccionadas_cv}")
print(f"Variables eliminadas (CV): {list(set(X_cov.columns) - set(variables_seleccionadas_cv))}")
print()

# 6. Comparar ambos métodos
print("=== COMPARACIÓN DE MÉTODOS ===")
print(f"Lambda validación cruzada: {lasso_cv.alpha_:.6f}")
print(f"Lambda Chernozhukov (para sklearn): {lambda_chernozhukov/n:.6f}")
print(f"Ratio (CV/Chernozhukov): {lasso_cv.alpha_/(lambda_chernozhukov/n):.2f}")
print()

print("Comparación de variables seleccionadas:")
print(f"Theórico: {variables_seleccionadas_theoretical}")
print(f"CV:       {variables_seleccionadas_cv}")
print(f"¿Coinciden? {set(variables_seleccionadas_theoretical) == set(variables_seleccionadas_cv)}")
print()

# 7. Re-estimar ATE con variables seleccionadas teóricas
if variables_seleccionadas_theoretical:
    formula_theoretical = 'Y ~ D + ' + ' + '.join(variables_seleccionadas_theoretical)
else:
    formula_theoretical = 'Y ~ D'

modelo_theoretical = smf.ols(formula_theoretical, data=df).fit()

print("=== REGRESIÓN CON SELECCIÓN TEÓRICA ===")
print("Fórmula:", formula_theoretical)
print("Coeficiente de D (ATE):", round(modelo_theoretical.params['D'], 3))
print("Error estándar:", round(modelo_theoretical.bse['D'], 3))
print()

# 8. También con selección de CV para comparar
if variables_seleccionadas_cv:
    formula_cv = 'Y ~ D + ' + ' + '.join(variables_seleccionadas_cv)
else:
    formula_cv = 'Y ~ D'

modelo_cv = smf.ols(formula_cv, data=df).fit()

print("=== REGRESIÓN CON SELECCIÓN CV ===")
print("Fórmula:", formula_cv)
print("Coeficiente de D (ATE):", round(modelo_cv.params['D'], 3))
print("Error estándar:", round(modelo_cv.bse['D'], 3))
print()

# 9. Comparación final
print("=== COMPARACIÓN FINAL DE ATE ===")
print(f"ATE verdadero: 2.000")
print(f"ATE teórico:   {modelo_theoretical.params['D']:.3f}")
print(f"ATE CV:        {modelo_cv.params['D']:.3f}")
print(f"ATE completo:  {modelo_completo.params['D']:.3f}")
print(f"ATE simple:    {modelo_simple.params['D']:.3f}")


3.3 LASSO CON PARÁMETROS TEÓRICOS (Enfoque Chernozhukov et al.)
Tamaño de muestra (n): 1000
Número de covariables (p): 4
Lambda teórico Chernozhukov (crudo): 86.882820
Lambda para sklearn (dividido por n): 0.086883

Ajustando LASSO con validación cruzada...
✓ LASSO con validación cruzada completado

=== RESULTADOS LASSO TEÓRICO ===
Coeficientes con lambda de Chernozhukov:
  Variable  Coeficiente  Abs_Coeficiente
0       X1     0.442946         0.442946
2       X3     0.162659         0.162659
1       X2    -0.002525         0.002525
3       X4    -0.000000         0.000000

Variables seleccionadas: ['X1', 'X3', 'X2']
Variables eliminadas: ['X4']

=== RESULTADOS VALIDACIÓN CRUZADA ===
Coeficientes con validación cruzada:
  Variable  Coeficiente  Abs_Coeficiente
0       X1     0.541919         0.541919
1       X2    -0.365257         0.365257
2       X3     0.258615         0.258615
3       X4    -0.067635         0.067635

Variables seleccionadas (CV): ['X1', 'X2', 'X3', 'X4']
Variable

C:\Users\fcdav\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_coordinate_descent.py:1622: FutureWarning: 'n_alphas' was deprecated in 1.7 and will be removed in 1.9. 'alphas' now accepts an integer value which removes the need to pass 'n_alphas'. The default value of 'alphas' will change from None to 100 in 1.9. Pass an explicit value to 'alphas' and leave 'n_alphas' to its default value to silence this warning.
  warnings.warn(
